In [2]:
### full data analysis and string assembly for HTML implementation ###

''' Point and Kill-based minigames, same regex pattern applies 
    1. Walls
    2. Mini SG 
    3. Mini SW
    4. Lasertag
    5. Mienengefecht
    6. OITC
    7. Paintball
    8. Spleef 
    9. Reihenfolge 
    10. Schießstand
    11. Sammelwahn
    12. Buntes Chaos
    13. Duelle 
'''

import os 
import re
import pandas as pd 
import pyperclip as pc

directory = "in/"
fileList = [os.path.join(directory, file) for file in os.listdir(directory)]

def getPlayerName(path):  
    name_pattern = re.compile(r"/([a-zA-Z0-9_]+)\.txt")
    match_name = name_pattern.search(path)

    # Extracted name
    extracted_name = match_name.group(1) if match_name else None
    
    return extracted_name

playerList = []

for file in fileList:
    player = getPlayerName(file)
    playerList.append(player)


def getPointRecord(filepath, minigame):
    records = []
    
    if type(filepath) == str:
        try: 
            fh = open(filepath, 'r')
        except: 
            print("Invalid Filepath")
            return None

        for line in fh:
            if minigame in line:
                player_name = getPlayerName(filepath)

                pattern = r"literal\{([^}]*)\}\[style=\{\}\], literal\{: \}\[style=\{\}\]\].+?literal\{(\d+)\}\[style=\{\}\]"
                matches = re.findall(pattern, line)         


                #for map_name, value in matches:
                    #if value != None:
                        #try: value = int(matches)
                        # except: print(value, "could not be converted to an integer")

                # Creating a dictionary from the matches
                data = {map_name: int(value) for map_name, value in matches}

                # Converting the dictionary into a DataFrame
                df = pd.DataFrame([data], index=[player_name])

                return df

        fh.close()


def getHtmlPointRecords(minigame):
    combined_df = pd.DataFrame()

    for filePath in fileList:
        if "txt" in filePath:
            temp_df = getPointRecord(filePath, minigame)
            combined_df = pd.concat([combined_df, temp_df], axis=0)
        
    return combined_df.fillna(0).   to_html().replace('\n', '').replace('<table border="1" class="dataframe">', '').replace('</table>', '').replace(".0<", "<")

def getDataFramePointRecords(minigame):
    combined_df = pd.DataFrame()

    for filePath in fileList:
        if "txt" in filePath:
            temp_df = getPointRecord(filePath, minigame)
            combined_df = pd.concat([combined_df, temp_df], axis=0)
            combined_df.fillna(0, inplace=True)

    return combined_df.fillna(0)   

'''
    Functions to enable compatibility with Time-based minigames (aside from JnR)
'''

def parseTime(minutes, seconds, milliseconds):
    # If minutes is None or empty, default to '0'
    if not minutes:
        minutes = '0'
    return f"{minutes}:{seconds}\"{milliseconds}"

def getMapRecord(filepath, minigame):
    if type(filepath) == str:
        try: 
            with open(filepath, 'r') as fh:
                file_content = fh.readlines()
        except: 
            print("Invalid Filepath")
            return None

        for line in file_content:
            if minigame in line:
                # Regex pattern for times greater than 1 minute
                pattern_gt_1min = r"literal\{([^}]*)\}\[style=\{\}\], literal\{: \}\[style=\{\}\].+?literal\{(\d+)\}\[style=\{\}\], literal\{min \}\[style=\{\}\], literal\{(\d+)\}\[style=\{\}\], literal\{s \}\[style=\{\}\], literal\{(\d+)\}\[style=\{\}\]"
                # Regex pattern for times less than 1 minute
                pattern_lt_1min = r"literal\{([^}]*)\}\[style=\{\}\], literal\{: \}\[style=\{\}\].+?literal\{(\d+)\}\[style=\{\}\], literal\{s \}\[style=\{\}\], literal\{(\d+)\}\[style=\{\}\]"
                
                # Try matching pattern for times greater than 1 minute
                matches = re.findall(pattern_gt_1min, line)
                if not matches:
                    # Try matching pattern for times less than 1 minute
                    matches = re.findall(pattern_lt_1min, line)
                    # Adjust match structure to fit parseTime function
                    matches = [(m[0], None, m[1], m[2]) for m in matches]

                if matches:
                    player_name = getPlayerName(filepath)
                    data = {map_name.strip(): parseTime(*time_parts) for map_name, *time_parts in matches}
                    df = pd.DataFrame([data], index=[player_name])
                    
                    if "rache" in minigame: return df.fillna('0:00"000')
                    return df
    return None

def GetJnRRecords(minigame, html=False):
    combined_df = getDataFrameTimeRecords(minigame) 

    canyon_df = combined_df[['Canyon']].copy() if 'Canyon' in combined_df.columns else pd.DataFrame()

    # modifing data in place to remove canyon after initial df has been created (canyon >1 min times)
    for filePath in fileList:
        if "txt" in filePath:
            with open(filePath, 'r') as file:
                file_data = file.read()

            file_data = file_data.replace('[literal{Canyon}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{1}[style={}], literal{min }[style={}], literal{', 'TEMP-REPLACEMENT')
            file_data = file_data.replace('[literal{Canyon}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{2}[style={}], literal{min }[style={}], literal{', 'TEMP-REPLACEMENT')

            with open(filePath, 'w') as file:
                file.write(file_data)

    # getting other map times 
    modified_df = getDataFrameTimeRecords(minigame)

    final_df = pd.concat([canyon_df, modified_df], axis=1)

    # restoring data
    for filePath in fileList:
        if "txt" in filePath:
            with open(filePath, 'r') as file:
                file_data = file.read()

            file_data = file_data.replace('TEMP-REPLACEMENT', '[literal{Canyon}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{1}[style={}], literal{min }[style={}], literal{')
            file_data = file_data.replace('TEMP-REPLACEMENT', '[literal{Canyon}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{2}[style={}], literal{min }[style={}], literal{')

            with open(filePath, 'w') as file:
                file.write(file_data)

    if html==False: return final_df
    if html==True: return final_df.to_html().replace('\n', '').replace('<table border="1" class="dataframe">', '').replace('</table>', '')

def GetWettrRecords(minigame='Wettr', html=False):
    combined_df = getDataFrameTimeRecords(minigame) # extract now (other columns won't work)

    nonSnow_df = combined_df[['Green Hills', 'Cherry Blossom Canyon', 'Cyberpunk']].copy() if 'Green Hills' in combined_df.columns else pd.DataFrame() # create df for treating other maps differently 

    # debug
    # return combined_df.sort_values(by='Cherry Blossom Canyon') # combined_df contains all records > 1min

    # modifing data in place to remove all > 1min times 
    for filePath in fileList:
        if "txt" in filePath:
            with open(filePath, 'r') as file:
                file_data = file.read()

            file_data = file_data.replace('siblings=[literal{Green Hills}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{1}[style={}], literal{min }[style={}], literal{', 'TEMP-GH-REPLACEMENT')
            file_data = file_data.replace('siblings=[literal{Cherry Blossom Canyon}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{1}[style={}], literal{min }[style={}], literal{', 'TEMP-CB-REPLACEMENT')
            file_data = file_data.replace('siblings=[literal{Cyberpunk}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{1}[style={}], literal{min }[style={}], literal{', 'TEMP-CP-REPLACEMENT')
            file_data = file_data.replace('siblings=[literal{Islands}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{1}[style={}], literal{min }[style={}], literal{', 'TEMP-IS-REPLACEMENT')

            with open(filePath, 'w') as file:
                file.write(file_data)

    # getting other map times 
    modified_df = getDataFrameTimeRecords(minigame)

    # debug
    # return modified_df

    final_df = pd.concat([nonSnow_df, modified_df], axis=1)

    # restoring data
    for filePath in fileList:
        if "txt" in filePath:
            with open(filePath, 'r') as file:
                file_data = file.read()

            file_data = file_data.replace('TEMP-GH-REPLACEMENT', 'siblings=[literal{Green Hills}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{1}[style={}], literal{min }[style={}], literal{')
            file_data = file_data.replace('TEMP-CB-REPLACEMENT', 'siblings=[literal{Cherry Blossom Canyon}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{1}[style={}], literal{min }[style={}], literal{')
            file_data = file_data.replace('TEMP-CP-REPLACEMENT', 'siblings=[literal{Cyberpunk}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{1}[style={}], literal{min }[style={}], literal{')
            file_data = file_data.replace('TEMP-IS-REPLACEMENT', 'siblings=[literal{Islands}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{1}[style={}], literal{min }[style={}], literal{')

            with open(filePath, 'w') as file:
                file.write(file_data)

    sub1minCBplayers = ['xBaumeisterin', 'LeWi_100', 'Fflopse']
    sub1minCBrecords = [['1:08"220', '1:06"491', '1:20:938', '0:45"625'], ['1:07"983', '0:57"947', '1:22"268', '0:46"461'], ['1:06"707', '0:56"032', '1:15"806', '0:44"748']]
    columns = ['Green Hills', 'Cherry Blossom Canyon', 'Cyberpunk', 'Snow']

    manual_df = pd.DataFrame(sub1minCBrecords, index=sub1minCBplayers, columns=columns)

    final_df = pd.concat([final_df, manual_df], axis=0)

    if html==False: return final_df
    if html==True: return final_df.to_html().replace('\n', '').replace('<table border="1" class="dataframe">', '').replace('</table>', '')


def getDataFrameTimeRecords(minigame):   
    combined_df = pd.DataFrame()

    for filePath in fileList:
        if "txt" in filePath:
            temp_df = getMapRecord(filePath, minigame)
            if temp_df is not None:
                combined_df = pd.concat([combined_df, temp_df], axis=0)

    return combined_df

def getHtmlTimeRecords(minigame):   
    combined_df = pd.DataFrame()

    for filePath in fileList:
        if "txt" in filePath:
            temp_df = getMapRecord(filePath, minigame)
            if temp_df is not None:
                combined_df = pd.concat([combined_df, temp_df], axis=0)
    
    if minigame in "Drachenflucht": return combined_df.fillna('0:00"000').to_html().replace('\n', '').replace('<table border="1" class="dataframe">', '').replace('</table>', '')
    if minigame in "Replika": return combined_df.to_html().replace('\n', '').replace('<table border="1" class="dataframe">', '').replace('</table>', '').replace('0:8"','0:08"').replace('0:7"','0:07"').replace('0:9"','0:09"').replace('0:6"','0:06"').replace('0:5"','0:05"').replace('0:4"','0:04"').replace('0:3"','0:03"').replace('0:2"','0:02"')
    else: return combined_df.to_html().replace('\n', '').replace('<table border="1" class="dataframe">', '').replace('</table>', '')

def getPointRecordRank(minigame, abs=False):
    if abs == True: return getDataFramePointRecords(minigame).rank(ascending=False, na_option='bottom', method='dense').sum(axis=1).sort_values()
    else: return getDataFramePointRecords(minigame).rank(ascending=False, na_option='bottom', method='dense').sum(axis=1).rank(method='min').sort_values()

miniGameList = ["1-Sammelwahn",
        "2-Schießstand",
        "3-Wettrennen",
        "4-Pferderennen",
        "5-Drachenflucht",
        "6-Jump and Run",
        "7-Hoch Hinaus",
        "8-Blockhüpfer",
        "9-Todeswürfel",
        "10-Freier Fall",
        "11-Elytrarennen",
        "12-Waffenfolge",
        "13-Minenfeld",
        "14-Kletterkönig",
        "15-Ampelrennen",
        "16-Replika",
        "17-Walls",
        "18-Mini SG",
        "19-Mini SW",
        "20-Lasertag",
        "21-Minengefecht",
        "22-OITC",
        "23-Paintball",
        "24-Spleef",
        "25-Buntes Chaos",
        "26-Reihenfolge",
        "27-Duelle"]

queryList = ["Sammel",
        "Schie",
        "Wettr",
        "Pferde",
        "Drache",
        "Jump",
        "Hoch",
        "Blockh",
        "Todesw",
        "Freie",
        "Elytra",
        "Waffe",
        "Minenf",
        "Kletter",
        "Ampelrennen",
        "Repli",
        "Walls",
        "Survival",
        "Mini Skyw",
        "Lasertag",
        "Mineng",
        " im ",
        "Paintball",
        "Spleef",
        "Chaos",
        "Reihe",
        "Duel"]

pointMiniGames = "Sammelwahn, Schießstand, Walls, Mini Survival Games, Mini Skywars, Lasertag, Minengefecht, Einer im Köcher, Paintball, Spleef, Buntes Chaos, Reihenfolge, Duelle" # contains Kill Minigames asw as they use the same functions 
timeMiniGames = "Pferderennen, Drachenflucht, Replika, Hoch Hinaus, Blockhüpfer, Todeswürfel, Freier Fall, Elytrarennen, Waffenfolge, Minenfeld, Kletterkönig, Ampelrennen" # the rest, but JnR and Wettrennen


frontWrapperList = []
x = 0 

for i in miniGameList: # assembling heads of table wrappers for readability and compatibility with the rest of the code. 
    import pyperclip as pc
    x = x + 1
    strX = str(x)
    frontWrapperList.append('<!-- Table: ' + i + ' --> <div id="table' + strX + '" style="display:none;"> <table id="dataframe' + strX + '" class="display nowrap">')

endWrapper = "</table> </div>"

def assembleHtmlString(query, i): # assemble HTML codes for desired Minigames
    if query in pointMiniGames:
        print(frontWrapperList[i])
        # print("query", query)             # debug
        print(getHtmlPointRecords(query))
        print(endWrapper)
    if query in timeMiniGames: 
        print(frontWrapperList[i])
        # print("query", query)             # debug
        print(getHtmlTimeRecords(query))
        print(endWrapper)
    if query in "Jump and Run":
        print(frontWrapperList[i])
        print(GetJnRRecords(query, html=True))
        print(endWrapper)
    if query in "Wettrennen":
        print(frontWrapperList[i])
        print(GetWettrRecords(html=True))
        print(endWrapper)
    if query not in timeMiniGames:
        if query not in pointMiniGames: 
            if query not in "Jump and Run":
                if query not in "Wettrennen":
                    raise ValueError("Query not in Supported Minigames (check language?)")

x = 0
outputString = ""

print('<div class="table-container">')
for i in queryList: 
    assembleHtmlString(query=i, i=x) 
    x = x + 1 
print('</div>')


<div class="table-container">
<!-- Table: 1-Sammelwahn --> <div id="table1" style="display:none;"> <table id="dataframe1" class="display nowrap">


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfc in position 456: invalid start byte

In [ ]:
GetWettrRecords(html=False).sort_values('Snow')

# GetWettrRecords(html=False).loc['Fflopse']

,Green Hills,Cherry Blossom Canyon,Cyberpunk,Snow
cediiiiii_10,"1:04""439","1:18""876","1:18""799","0:44""679"
Fflopse,"1:06""707","0:56""032","1:15""806","0:44""748"
lizsyy,"1:05""912","1:20""152","1:20""763","0:44""964"
HerrDante,"1:04""999","1:20""568","1:20""245","0:45""063"
Grafikkatze,"1:08""698","1:19""685","1:22""396","0:45""099"
...,...,...,...,...
Kokochampi,"1:10""614","1:33""566","1:31""396","0:51""934"
Chander24,"1:12""305","1:27""484","1:26""495","0:52""084"
ForceFox,"1:10""643","1:29""247","1:26""398","0:53""071"
Bartschii,"1:15""910",NaN,NaN,"0:53""246"


In [ ]:
sub1minCBplayers = ['xBaumeisterin', 'LeWi_100', 'Fflopse']
sub1minCBrecords = [['1:08"220', '1:06"491', '1:20:938', '0:45"625'], ['1:07"983', '57"947', '1:22"268', '46"461'], ['1:06"707', '56"032', '1:15"806', '44"748']]
columns = ['Green Hills', 'Cherry Blossom Canyon', 'Cyberpunk', 'Snow']

df = pd.DataFrame(sub1minCBrecords, index=sub1minCBplayers, columns=columns)
df

,Green Hills,Cherry Blossom Canyon,Cyberpunk,Snow
xBaumeisterin,"1:08""220","1:06""491",1:20:938,"0:45""625"
LeWi_100,"1:07""983","57""947","1:22""268","46""461"
Fflopse,"1:06""707","56""032","1:15""806","44""748"


In [ ]:
getPointRecordRank("Sammelwahn", abs=True).head(10)

cediiiiii_10     83.0
HerrDante       110.0
Gerrygames      112.0
lizsyy          129.0
KakaMC          136.0
Grapfen         139.0
DarkCobweb      142.0
Kiritoonroad    158.0
Sey__           172.0
Lubotter2009    174.0
dtype: float64

In [ ]:
getPointRecordRank("Lasert", abs=True).head(10)

HerrDante        4.0
Kiritoonroad     5.0
cediiiiii_10    13.0
_n3d            21.0
Grapfen         21.0
h4nnes          22.0
lizsyy          23.0
Gerrygames      23.0
frutigall       24.0
tsuneee         24.0
dtype: float64

In [ ]:
getPointRecordRank("Einer", abs=True).head(10)

HerrDante         4.0
cediiiiii_10      8.0
Kiritoonroad     11.0
h4nnes           13.0
Grapfen          13.0
Brilux           18.0
Gummibearchen    20.0
tsuneee          22.0
byTobi           22.0
lizsyy           25.0
dtype: float64

In [ ]:
getDataFramePointRecords("Laser").sort_values(by='Port', ascending=False)

,Neon,Port,Factory
Kiritoonroad,43.0,42.0,65.0
HerrDante,50.0,40.0,88.0
Grapfen,40.0,38.0,42.0
frutigall,33.0,38.0,47.0
_n3d,37.0,34.0,47.0
...,...,...,...
Sebi1801,32.0,0.0,33.0
juvona,0.0,0.0,0.0
Kyuudo,0.0,0.0,0.0
LeWi_100,0.0,0.0,0.0
